Binary data transfer
====================

### Motivation

Often for visualizations in genomics, massive social networks, or sensor data visualizations,
it helps to be able to plot millions rather than simply hundreds of thousands of points.

By default, pydeck sends data from Jupyter to the frontend by serializing data to JSON. However, for massive data sets,
the costs to serialize and deserialize this JSON can prevent a visualization from rendering.

In order to get around this, pydeck supports binary data transfer, which significantly reduces data size. Binary transfer relies
on [NumPy](https://numpy.org/) and its [typed arrays](https://numpy.org/devdocs/user/basics.types.html),
which are converted to [JavaScript typed arrays](https://developer.mozilla.org/en-US/docs/Web/JavaScript/Typed_arrays) and passed to
deck.gl [using precalculated binary attributes](https://deck.gl/#/documentation/developer-guide/performance-optimization?section=supply-attributes-directly).



### Usage

Binary transport will only work if the following requirements are met:

-  **Use Jupyter.** Binary transfer only works within Jupyter environments via `pydeck.bindings.deck.Deck.show`. It relies on the socket-level
  communication built into the Jupyter environment.
- **Set ``use_binary_transport``.** ``use_binary_transport`` must be set to ``True`` explictly on your ``Layer``.
- **Pass data as a DataFrame.** The ``Layer``'s ``data`` argument must be a ``pandas.DataFrame`` object.
- **Don't pass un-used columns.** Data that is not intend to be rendered should not be passed into the ``Layer``.
- **Prefer columns of lists where possible**. Accessor names must be strings representing column names within the ``pandas.DataFrame``. For example, ``get_position='position'`` is correct, **not** ``get_position=['x', 'y']``.
  For example, this data format, where ``x`` & ``y`` represent a position and ``r``, ``g``, and ``b`` represent color values, like this–

| x | y |  r  |  g  | b |
|---|:--|:----|:----|:--|
| 0 | 1 |  0  |  0  | 0 |
| 0 | 5 | 255 |  0  | 0 |
| 5 | 1 | 255 | 255 | 0 |

should be converted to a nested format like this–

| position   | color         |
|------------|:--------------|
| [0, 1]     | [0, 0, 0]     |
| [0, 5]     | [255, 0, 0]   |
| [5, 1]     | [255, 255, 0] |

## Example

To demonstrate this, we'll plot a series of nodes generated at random from the [networkx](https://networkx.github.io/) library:

In [ ]:
import pydeck as pdk
import pandas as pd


NODES_URL = "https://raw.githubusercontent.com/ajduberstein/geo_datasets/master/social_nodes.csv"


def generate_graph_data(num_nodes, random_seed):
    """Generates a graph of 10k nodes with a 3D force layout

    This function is unused but serves as an example of how the data in
    this visualization was generated
    """
    import networkx as nx  # noqa

    g = nx.random_internet_as_graph(num_nodes, random_seed)
    node_positions = nx.fruchterman_reingold_layout(g, dim=3)

    force_layout_df = pd.DataFrame.from_records(node_positions).transpose()
    force_layout_df["group"] = [d[1]["type"] for d in g.nodes.data()]
    force_layout_df.columns = ["x", "y", "z", "group"]
    return force_layout_df

We will use deck.gl's OrbitView (example [here](https://deck.gl/gallery/point-cloud-layer)) which can help us plot these points in 3D:

The data we'll render is a 3D position with some categorical data (``group``):

In [ ]:
nodes = pd.read_csv(NODES_URL)
nodes.head()

We need this flattened data in a nested format, so we transform it:

In [ ]:
nodes["position"] = nodes.apply(lambda row: [row["x"], row["y"], row["z"]], axis=1)

# Remove all unused columns
del nodes["x"]
del nodes["y"]
del nodes["z"]
nodes.head()

We'll also color the points by group and then normalize the colors since they are provided as floats–see the [deck.gl documentation](https://deck.gl/docs/developer-guide/performance#supply-attributes-directly).

In [ ]:
colors = pdk.data_utils.assign_random_colors(nodes["group"])
# Divide by 255 to normalize the colors
# Specify positions and colors as columns of lists
nodes["color"] = nodes.apply(
    lambda row: [c / 255 for c in colors.get(row["group"])], axis=1
)
# Remove unused column
del nodes["group"]
nodes.head()

In [ ]:
view_state = pdk.ViewState(
    offset=[0, 0],
    latitude=None,
    longitude=None,
    bearing=None,
    pitch=None,
    zoom=10
)
views = [pdk.View(type="OrbitView", controller=True)]

nodes_layer = pdk.Layer(
    "PointCloudLayer",
    nodes,
    id="binary-points",
    use_binary_transport=True,
    get_position="position",
    get_normal=[10, 100, 10],
    get_color="color",
    pickable=True,
    auto_highlight=True,
    highlight_color=[255, 255, 0],
    radius=50,
)

pdk.Deck(layers=[nodes_layer], initial_view_state=view_state, views=views, map_provider=None).show()

### Confirming that data is being transferred over web sockets

You can open up your brower's developer console to confirm the data is being sent via binary transfer. In the console you should see a message like:

    >> transport.js:68 Delivering transport message 
    >> binary: {"binary-points": {…}}
    >> transport: n {name: "Jupyter Transport (JavaScript <=> Jupyter Kernel)", _messageQueue: Array(0), userData: {…}, jupyterModel: n, jupyterView: n}
    type: "json-with-binary"

You should also be able to see the binary payload in your developer console's Network tab.